In [102]:
import pandas as pd
import numpy as np

import datetime


import pickle

from itertools import chain

from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV, LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

import pickle

In [103]:
train = pd.read_csv('../../data/engineered/training.csv', index_col='id')
submission_input = pd.read_csv('../../data/engineered/test.csv', index_col='id')

target = 'SalePrice'
y = train[target]
X = train[[col for col in train.columns if col != target]]
y_log = np.log1p(y)

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=.5, random_state=13)

In [105]:
n_iter = 1000
alphas = np.random.exponential(1, 100)

lasso_model = LassoCV(alphas=alphas, max_iter=n_iter)
lasso_model.fit(X_train, y_train)

LassoCV(alphas=array([ 0.51785,  1.79934, ...,  0.34345,  0.34245]),
    copy_X=True, cv=None, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [106]:
time_lasso = datetime.datetime.now()

In [107]:
lasso_model.score(X_test, y_test)

0.82731237684544745

In [108]:
mean_squared_error(y_test, lasso_model.predict(X_test))**.5

0.17097865484528468

In [109]:
mean_squared_error(np.exp(y_test) - 1, np.exp(lasso_model.predict(X_test)) - 1)**.5

33220.791747237359

In [110]:
with open('lasso.pickle', "wb") as pickle_model:
    pickle.dump(lasso_model, pickle_model)


# with open('lasso.pickle', "rb") as pickle_model:
#     load_model = pickle.load(pickle_model)

In [114]:
lasso_pred = lasso_model.predict(submission_input)
lasso_pred = np.exp(lasso_pred) - 1
submission = pd.DataFrame(data=
                         {
                             'SalePrice':lasso_pred
                         },
                         index=submission_input.index)
submission.sort_index().to_csv('lasso_submission.csv')

In [11]:
alphas = np.random.exponential(1, 100)

ridge_model = RidgeCV(alphas=alphas, cv=4)
ridge_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.37173,  0.65108, ...,  0.04902,  1.59189]), cv=4,
    fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [12]:
ridge_model.score(X_test, y_test)

0.81009524511939168

In [13]:
mean_squared_error(y_test, ridge_model.predict(X_test))**.5

0.17213326222770967

In [14]:
mean_squared_error(np.exp(y_test) - 1, np.exp(ridge_model.predict(X_test)) - 1)**.5

34767.884625352592

In [15]:
with open('ridge.pickle', "wb") as pickle_model:
    pickle.dump(ridge_model, pickle_model)


# with open('ridge.pickle', "rb") as pickle_model:
#     load_model = pickle.load(pickle_model)

In [118]:
ridge_pred = ridge_model.predict(submission_input)
ridge_pred = np.exp(ridge_pred) + 1
ridge_pred = pd.DataFrame(data=
                         {
                             'SalePrice':ridge_pred
                         },
                         index=submission_input.index)
submission.sort_index().to_csv('ridge_submission.csv')

In [122]:
l1_ratio = [*chain(np.linspace(.01, .3, 10), np.linspace(.6,1, 90))]
max_iter = 1000

elastic_model = ElasticNetCV(l1_ratio=l1_ratio, max_iter=max_iter, cv=5)
elastic_model.fit(X_train, y_train)
elastic_model.score(X_test, y_test)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.83400704414961424

In [129]:
elastic_model.score(X_test, y_test)

0.83400704414961424

In [130]:
mean_squared_error(y_test, elastic_model.predict(X_test))**.5

0.16763168839263196

In [131]:
mean_squared_error(np.exp(y_test) - 1, np.exp(elastic_model.predict(X_test)) - 1)**.5

33067.54218455917

In [132]:
with open('ridge.pickle', "wb") as pickle_model:
    pickle.dump(ridge_model, pickle_model)


# with open('ridge.pickle', "rb") as pickle_model:
#     load_model = pickle.load(pickle_model)

In [133]:
elastic_pred = elastic_model.predict(submission_input)
elastic_pred = np.exp(elastic_pred) - 1

submission = pd.DataFrame(data=
                         {
                             'SalePrice':elastic_pred
                         },
                         index=submission_input.index)
submission.sort_index().to_csv('elastic_submission.csv')

In [134]:
time_elastic = datetime.datetime.now()